# Intro to Logging with Python

# What You Will Learn

* Creating a log
* Logging Levels
* Logging Handlers
* Logging Formatters
* Logging to Multiple Locations
* and more!

# Creating a Log

In [ ]:
import logging

logging.warning('This is a warning message')

# Logging Levels

## Logging levels from lowest to highest

* DEBUG - Detailed information, typically of interest only when diagnosing problems.

* INFO - Confirmation that things are working as expected.

* WARNING - An indication that something unexpected happened  (default level)

* ERROR - A serious error has occurred
	
* CRITICAL - The program has stopped or will stop


The lower the logging level, the more logging levels will be logged

In [ ]:
import logging

logging.warning('This is a warning message')
logging.info('This is an information message')  # this one won't work

In [ ]:
import logging

logging.info('This is an information message')  # this one won't work
logging.warning('This is a warning message')
logging.error('An error has occurred')

# Setting the Logging Level

In [ ]:
# setting_log_level.py

import logging

# add filemode="w" to overwrite
logging.basicConfig(filename="test.log", level=logging.INFO)

logging.debug("This is a debug message") # This one won't log
logging.info("Informational message")
logging.error("An error has happened!")

# Creating a Logger Object

In [ ]:
# create_logger.py

import logging

logging.basicConfig(filename="ex_logger.log", level=logging.INFO)
log = logging.getLogger("ex")

try:
    raise RuntimeError
except Exception, err:
    log.exception("Error!")

# Logging Handlers

# Handler Types

* StreamHandler - stdout, stderr or file-like objects
* FileHandler - for writing to disk
* RotatingFileHandler - supports log rotation
* TimedRotatingFileHandler - supports rotation of disk log files at certain timed intervals


* SocketHandler - sends logging output to a network socket
* SMTPHandler - supports sending logging messages to an email address via SMTP

More handlers can be found here: https://docs.python.org/3/library/logging.handlers.html#module-logging.handlers

In [ ]:
# StreamHandler - stream_log.py

import logging

logger = logging.getLogger('stream_logger')
logger.setLevel(logging.INFO)

console = logging.StreamHandler()

logger.addHandler(console)
logger.info("This is an informational message")

In [ ]:
# SMTPHandler - email_log.py

import logging
import logging.handlers

logger = logging.getLogger("email_logger")
logger.setLevel(logging.INFO)
fh = logging.handlers.SMTPHandler('smtp.my_server.com',
                                  fromaddr='log@my_server.com',
                                  toaddrs=['mike@my_server.com'],
                                  subject='Python log')
logger.addHandler(fh)
logger.info("This is an informational message")

# Logging Formatters

Formatters allow you to add formatting to the log messages

https://docs.python.org/3/library/logging.html#logrecord-attributes

# Formatter example


```python
formatter = logging.Formatter('%(asctime)s - %(name)s - %(message)s')
```

## Sample Output

```
2020-02-28 08:44:14,902 - LOGGER_NAME - This is an informational message
```

LogRecord attributes:

https://docs.python.org/3/library/logging.html#logrecord-attributes

In [ ]:
# log_formatter.py

import logging

logger = logging.getLogger('stream_logger')
logger.setLevel(logging.INFO)

console = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s - %(name)s - %(message)s')
console.setFormatter(formatter)

logger.addHandler(console)
logger.info("This is an informational message")

The `logging` module is thread-safe

Logging may not work in asynchronous contexts

But you can use the QueueHandler as a workaround

https://docs.python.org/3/library/logging.handlers.html#queuehandler

https://www.zopatista.com/python/2019/05/11/asyncio-logging/

# Logging to Multiple Handlers

In [ ]:
import logging
import os

def log(path, multipleLocs=False):
    """
    Log to multiple locations if multipleLocs is True
    """
    fname = os.path.splitext(path)[0]
    logger = logging.getLogger("Test_logger_%s" % fname)
    logger.setLevel(logging.INFO)
    fh = logging.FileHandler(path)
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(message)s')
    fh.setFormatter(formatter)
    logger.addHandler(fh)

    if multipleLocs:
        console = logging.StreamHandler()
        console.setLevel(logging.INFO)
        console.setFormatter(formatter)
        logger.addHandler(console)

    logger.info("This is an informational message")
    try:
        1 / 0
    except ZeroDivisionError:
        logger.exception("You can't do that!")

    logger.critical("THIS IS A SHOW STOPPER!!!")

# Logging Exceptions

```python
try:
    1 / 0
except ZeroDivisionError:
    logger.exception("You can't do that!")
```

```
2020-03-02 11:51:14,698 - Test_logger_sample2 - You can't do that!
Traceback (most recent call last):
  File "/Users/michael/Dropbox/pyowa/talks/log_multiple_locations.py", line 24, in log
    1 / 0
```

# Configuring Logging

* https://docs.python.org/3/library/logging.config.html
* fileConfig
* dictConfig

# Using a fileConfig

In [ ]:
[loggers]
keys=root,exampleApp

[handlers]
keys=fileHandler, consoleHandler

[formatters]
keys=myFormatter

[logger_root]
level=CRITICAL
handlers=consoleHandler

[logger_exampleApp]
level=INFO
handlers=fileHandler
qualname=exampleApp

[handler_consoleHandler]
class=StreamHandler
level=DEBUG
formatter=myFormatter
args=(sys.stdout,)

[handler_fileHandler]
class=FileHandler
formatter=myFormatter
args=("config.log",)

[formatter_myFormatter]
format=%(asctime)s - %(name)s - %(levelname)s - %(message)s
datefmt=

In [ ]:
# logging_with_config_file.py

import logging
import logging.config

logging.config.fileConfig('logging.conf')
logger = logging.getLogger("exampleApp")

logger.info("Program started")
logger.info("Done!")

# Using a dictConfig

In [ ]:
import logging
import logging.config

dictLogConfig = {
    "version":1,
    "handlers":{
                "fileHandler":{
                    "class":"logging.FileHandler",
                    "formatter":"myFormatter",
                    "filename":"dict_config.log"
                    }
                },
    "loggers":{
        "exampleApp":{
            "handlers":["fileHandler"],
            "level":"INFO",
            }
        },

    "formatters":{
        "myFormatter":{
            "format":"%(asctime)s - %(name)s - %(levelname)s - %(message)s"
            }
        }
    }

In [ ]:
logging.config.dictConfig(dictLogConfig)

logger = logging.getLogger("exampleApp")

logger.info("Program started")
logger.info("Done!")

# Rotating Logs

In [ ]:
import logging
import time
from logging.handlers import RotatingFileHandler

def create_rotating_log(path):
    """
    Creates a rotating log
    """
    logger = logging.getLogger("Rotating Log")
    logger.setLevel(logging.INFO)

    # add a rotating handler
    handler = RotatingFileHandler(path, maxBytes=20,
                                  backupCount=5)
    logger.addHandler(handler)

    for i in range(10):
        logger.info("This is test log line %s" % i)
        time.sleep(1.5)


if __name__ == "__main__":
    log_file = "rotated.log"
    create_rotating_log(log_file)

# Creating a Logging Decorator

https://www.blog.pythonlibrary.org/2016/06/09/python-how-to-create-an-exception-logging-decorator/

# Other Logging Packages

* structlog - http://www.structlog.org/en/stable/

# Questions?